<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#imports-and-functions" data-toc-modified-id="imports-and-functions-1">imports and functions</a></span></li><li><span><a href="#experiments" data-toc-modified-id="experiments-2">experiments</a></span><ul class="toc-item"><li><span><a href="#0" data-toc-modified-id="0-2.1">0</a></span></li><li><span><a href="#1" data-toc-modified-id="1-2.2">1</a></span></li><li><span><a href="#2" data-toc-modified-id="2-2.3">2</a></span></li><li><span><a href="#3" data-toc-modified-id="3-2.4">3</a></span></li></ul></li></ul></div>

# imports and functions

In [1]:
import collections
import pandas as pd
import os
import sys
import json
import matplotlib.pyplot as plt
from tqdm.auto import tqdm, trange

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import optim
from torch.optim import lr_scheduler
# import torchmetrics

import datasets
from datasets import load_metric
from transformers import AutoConfig, AutoTokenizer, BertModel, RobertaModel
from transformers import BertForSequenceClassification
from transformers import TrainingArguments, Trainer

from sklearn.metrics import mean_squared_error, accuracy_score, precision_recall_fscore_support


In [2]:
# https://github.com/huggingface/transformers/issues/5486
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
with open('../../data/xslue/tasks.json', 'r') as f:
    tasks = json.load(f)
tasks

{'CrowdFlower': 13,
 'DailyDialog': 7,
 'EmoBank_Valence': 1,
 'EmoBank_Arousal': 1,
 'EmoBank_Dominance': 1,
 'HateOffensive': 3,
 'PASTEL_age': 8,
 'PASTEL_country': 2,
 'PASTEL_education': 10,
 'PASTEL_ethnic': 10,
 'PASTEL_gender': 3,
 'PASTEL_politics': 3,
 'PASTEL_tod': 5,
 'SARC': 2,
 'SarcasmGhosh': 2,
 'SentiTreeBank': 1,
 'ShortHumor': 2,
 'ShortJokeKaggle': 2,
 'ShortRomance': 2,
 'StanfordPoliteness': 1,
 'TroFi': 2,
 'VUA': 2}

In [8]:
class MyDataset(Dataset): 
    # currently it's a Mapping-style dataset. Not sure if a Iterable-style dataset will be better
    def __init__(self, tsv_file):
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        self.df = pd.read_csv(tsv_file, sep='\t')
        self.df = self.df.dropna()
        self.df = self.df.reset_index(drop=True)
        self.encodings = self.tokenizer(self.df['text'].tolist(), truncation=True, padding=True, max_length=128)
        if self.df['label'].dtype == 'float64':
            self.df['label'] = self.df['label'].astype('float32')
        self.labels = self.df['label'].tolist()
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item


In [9]:
pearsonr = load_metric("pearsonr")
spearmanr = load_metric("spearmanr")

In [10]:
# GPU memory usage: 6617 - 6680mb with bs 32
# bs 64 gives OOM
# bs 48 GPU memory 7894
batch_size = 32

In [12]:
def train_baseline(task, freeze_bert=False):
    torch.cuda.empty_cache()
    model = None
    trainer = None 
    num_labels = tasks[task]
    
    data_folder = '../../data/xslue'
    train_dataset = MyDataset(f'{data_folder}/processed/train/{task}.tsv')
    test_dataset = MyDataset(f'{data_folder}/processed/test/{task}.tsv')
    valid_dataset = MyDataset(f'{data_folder}/processed/dev/{task}.tsv')
    
    singletaskbert = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels) 
    if freeze_bert:
        for param in singletaskbert.bert.parameters():
            param.requires_grad = False
    result_folder = '../../result'
    training_args = TrainingArguments(
        output_dir=f"{result_folder}/baselines/{task+'_freezed' if freeze_bert else task}",   # output directory
        num_train_epochs=5,              # total number of training epochs
        per_device_train_batch_size=batch_size,  # batch size per device during training
        per_device_eval_batch_size=batch_size,   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir=f"{result_folder}/baselines/{task+'_freezed' if freeze_bert else task}/logs",  # directory for storing logs
#         logging_first_step = True, 
#         logging_steps=500,               # log & save weights each logging_steps
#         save_steps=500,
        evaluation_strategy="epoch",     # evaluate each `logging_steps`
        save_total_limit = 1,
        save_strategy = 'epoch',
        load_best_model_at_end=True, # decide on loss
    )
    
    if num_labels == 1:
        def compute_metrics(pred):
            predictions, labels = pred
            rmse = mean_squared_error(labels, predictions, squared=False)
            return {"rmse": rmse}
    elif num_labels == 2:
        def compute_metrics(pred):
            labels = pred.label_ids
            preds = pred.predictions.argmax(-1)
            precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
            acc = accuracy_score(labels, preds)
            return {
                'accuracy': acc,
                'f1': f1,
                'precision': precision,
                'recall': recall
            }
    else:
        def compute_metrics(pred):
            labels = pred.label_ids
            preds = pred.predictions.argmax(-1)
            precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
            acc = accuracy_score(labels, preds)
            return {
                'accuracy': acc,
                'f1': f1,
                'precision': precision,
                'recall': recall
            }
    
    trainer = Trainer(
        model=singletaskbert,   # the instantiated Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training dataset
        eval_dataset=valid_dataset,          # evaluation dataset
#         test_dataset=test_dataset,            # test dataset
        compute_metrics=compute_metrics,     # the callback that computes metrics of interest
    )
    trainer.train()

# experiments

## 0

In [13]:
task = list(tasks.keys())[0]
train_baseline(task)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.833100,1.836723,0.374492,0.167900,0.268430,0.185773
2,1.660100,1.801839,0.391260,0.209050,0.244737,0.212823
3,1.344600,1.959348,0.380589,0.228437,0.245589,0.232396
4,0.954800,2.301294,0.350102,0.210632,0.214547,0.212391
5,0.676100,2.548324,0.345020,0.205854,0.207991,0.206892


***** Running Evaluation *****
  Num examples = 1968
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ../../result/baselines/CrowdFlower/checkpoint-1126
Configuration saved in ../../result/baselines/CrowdFlower/checkpoint-1126/config.json
Model weights saved in ../../result/baselines/CrowdFlower/checkpoint-1126/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1968
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this b

In [14]:
task = list(tasks.keys())[0]
train_baseline(task, freeze_bert=True)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/j

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABE

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,2.150600,2.149744,0.232724,0.048777,0.043375,0.086117
2,2.122500,2.136429,0.254573,0.055350,0.039144,0.094482
3,2.111200,2.124345,0.260671,0.057573,0.048524,0.096778
4,2.102000,2.119663,0.260163,0.057517,0.046000,0.096546
5,2.103800,2.118026,0.262195,0.057974,0.047341,0.097301


***** Running Evaluation *****
  Num examples = 1968
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ../../result/baselines/CrowdFlower_freezed/checkpoint-1126
Configuration saved in ../../result/baselines/CrowdFlower_freezed/checkpoint-1126/config.json
Model weights saved in ../../result/baselines/CrowdFlower_freezed/checkpoint-1126/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1968
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` par

## 1

In [15]:
task = list(tasks.keys())[1]
train_baseline(task)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/j

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.395000,0.280152,0.895402,0.341502,0.359248,0.338107
2,0.323800,0.257652,0.906432,0.360972,0.390918,0.345662
3,0.243700,0.302913,0.894783,0.449316,0.518872,0.429394
4,0.173100,0.351899,0.894535,0.389802,0.543825,0.368605
5,0.128200,0.411256,0.889330,0.403815,0.448151,0.390719


***** Running Evaluation *****
  Num examples = 8069
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ../../result/baselines/DailyDialog/checkpoint-2725
Configuration saved in ../../result/baselines/DailyDialog/checkpoint-2725/config.json
Model weights saved in ../../result/baselines/DailyDialog/checkpoint-2725/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 8069
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this b

In [16]:
task = list(tasks.keys())[1]
train_baseline(task, freeze_bert=True)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/j

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.596600,0.462708,0.880902,0.133812,0.125843,0.142857
2,0.577300,0.445178,0.880902,0.133812,0.125843,0.142857
3,0.556800,0.437267,0.880778,0.133801,0.125841,0.142837
4,0.553300,0.431007,0.880654,0.133791,0.125839,0.142817
5,0.551800,0.426904,0.880654,0.133791,0.125839,0.142817


***** Running Evaluation *****
  Num examples = 8069
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ../../result/baselines/DailyDialog_freezed/checkpoint-2725
Configuration saved in ../../result/baselines/DailyDialog_freezed/checkpoint-2725/config.json
Model weights saved in ../../result/baselines/DailyDialog_freezed/checkpoint-2725/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 8069
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` par

## 2

In [17]:
task = list(tasks.keys())[2]
train_baseline(task)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/j

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file

Epoch,Training Loss,Validation Loss,Rmse
1,No log,0.007214,0.084937
2,0.038700,0.007140,0.084501
3,0.038700,0.006620,0.081361
4,0.004100,0.005645,0.075130
5,0.004100,0.005498,0.074148


***** Running Evaluation *****
  Num examples = 498
  Batch size = 32
Saving model checkpoint to ../../result/baselines/EmoBank_Valence/checkpoint-276
Configuration saved in ../../result/baselines/EmoBank_Valence/checkpoint-276/config.json
Model weights saved in ../../result/baselines/EmoBank_Valence/checkpoint-276/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 498
  Batch size = 32
Saving model checkpoint to ../../result/baselines/EmoBank_Valence/checkpoint-552
Configuration saved in ../../result/baselines/EmoBank_Valence/checkpoint-552/config.json
Model weights saved in ../../result/baselines/EmoBank_Valence/checkpoint-552/pytorch_model.bin
Deleting older checkpoint [../../result/baselines/EmoBank_Valence/checkpoint-276] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 498
  Batch size = 32
Saving model checkpoint to ../../result/baselines/EmoBank_Valence/checkpoint-828
Configuration saved in ../../result/baselines/EmoBank_Valence/checkp

In [18]:
task = list(tasks.keys())[2]
train_baseline(task, freeze_bert=True)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/j

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file

Epoch,Training Loss,Validation Loss,Rmse
1,No log,0.023580,0.153557
2,0.130400,0.014939,0.122226
3,0.130400,0.013498,0.116180
4,0.024100,0.012525,0.111916
5,0.024100,0.012770,0.113003


***** Running Evaluation *****
  Num examples = 498
  Batch size = 32
Saving model checkpoint to ../../result/baselines/EmoBank_Valence_freezed/checkpoint-276
Configuration saved in ../../result/baselines/EmoBank_Valence_freezed/checkpoint-276/config.json
Model weights saved in ../../result/baselines/EmoBank_Valence_freezed/checkpoint-276/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 498
  Batch size = 32
Saving model checkpoint to ../../result/baselines/EmoBank_Valence_freezed/checkpoint-552
Configuration saved in ../../result/baselines/EmoBank_Valence_freezed/checkpoint-552/config.json
Model weights saved in ../../result/baselines/EmoBank_Valence_freezed/checkpoint-552/pytorch_model.bin
Deleting older checkpoint [../../result/baselines/EmoBank_Valence_freezed/checkpoint-276] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 498
  Batch size = 32
Saving model checkpoint to ../../result/baselines/EmoBank_Valence_freezed/checkpoint-828
Conf

## 3

In [19]:
task = list(tasks.keys())[3]
train_baseline(task)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/j

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file

Epoch,Training Loss,Validation Loss,Rmse
1,No log,0.007646,0.087439
2,0.037500,0.006880,0.082945
3,0.037500,0.007955,0.089189
4,0.006000,0.006539,0.080866
5,0.006000,0.007301,0.085444


***** Running Evaluation *****
  Num examples = 498
  Batch size = 32
Saving model checkpoint to ../../result/baselines/EmoBank_Arousal/checkpoint-276
Configuration saved in ../../result/baselines/EmoBank_Arousal/checkpoint-276/config.json
Model weights saved in ../../result/baselines/EmoBank_Arousal/checkpoint-276/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 498
  Batch size = 32
Saving model checkpoint to ../../result/baselines/EmoBank_Arousal/checkpoint-552
Configuration saved in ../../result/baselines/EmoBank_Arousal/checkpoint-552/config.json
Model weights saved in ../../result/baselines/EmoBank_Arousal/checkpoint-552/pytorch_model.bin
Deleting older checkpoint [../../result/baselines/EmoBank_Arousal/checkpoint-276] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 498
  Batch size = 32
Saving model checkpoint to ../../result/baselines/EmoBank_Arousal/checkpoint-828
Configuration saved in ../../result/baselines/EmoBank_Arousal/checkp

In [20]:
task = list(tasks.keys())[3]
train_baseline(task, freeze_bert=True)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/j

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file

Epoch,Training Loss,Validation Loss,Rmse
1,No log,0.018002,0.134173
2,0.121600,0.011728,0.108297
3,0.121600,0.010726,0.103565
4,0.022400,0.010309,0.101535
5,0.022400,0.010187,0.100931


***** Running Evaluation *****
  Num examples = 498
  Batch size = 32
Saving model checkpoint to ../../result/baselines/EmoBank_Arousal_freezed/checkpoint-276
Configuration saved in ../../result/baselines/EmoBank_Arousal_freezed/checkpoint-276/config.json
Model weights saved in ../../result/baselines/EmoBank_Arousal_freezed/checkpoint-276/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 498
  Batch size = 32
Saving model checkpoint to ../../result/baselines/EmoBank_Arousal_freezed/checkpoint-552
Configuration saved in ../../result/baselines/EmoBank_Arousal_freezed/checkpoint-552/config.json
Model weights saved in ../../result/baselines/EmoBank_Arousal_freezed/checkpoint-552/pytorch_model.bin
Deleting older checkpoint [../../result/baselines/EmoBank_Arousal_freezed/checkpoint-276] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 498
  Batch size = 32
Saving model checkpoint to ../../result/baselines/EmoBank_Arousal_freezed/checkpoint-828
Conf